In [1]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import scipy.interpolate
import statistics
import seaborn as sns
import os
import statsmodels.api as sm
from numpy import asarray
from numpy import savetxt
import pickle
import csv
import numpy as np
from scipy import stats


In [2]:
#Obrim els seguents archius:
#Data EC amb 12 brain reagions y els powerband
with open("/Users/laianavarroolivella/Proyectos/EEG/2.banpowerEpoch12BR/list_finalEC.txt", "rb") as fp: 
    dataEC = pickle.load(fp)

#Llista dels noms dels pacients EC 
with open("/Users/laianavarroolivella/Proyectos/EEG/1.dataToBrainReagions/sorted_list_EC.txt", "rb") as fp:   # Unpickling
    listEC_Names = pickle.load(fp)

In [3]:
lis_noms = ['Anterior_midline', 'Left_frontal', 'Right_frontal', 'Left_temporal', 'Left_central', 'Left_parietal', 'Left_Occipital', 'Right_Occipital', 'Right_parietal', 'Right_temporal', 'Posterior_midline', 'Right_central']

labels = ['Delta', 'Theta', 'Alpha', 'Beta', 'Gama Lower']

In [4]:
dabeNames_BR_Bads_BP =[]
for e in range(0, len(dataEC)):  #Per cada brain reagon
    dp = []
    for i in range(0, len(dataEC[e])): #Per cada Band
        new =[]
        for r in listEC_Names:
            new.append(str(r))
        dp.append(new)
    dabeNames_BR_Bads_BP.append(dp)


In [5]:
#Z-Score, out the outliers
Clean_Data = [] #On guardarem totes les dades [pacients][bands][data]
New_listnames = [] #On guararem els noms dels pacients que han passat
Possitions = [] #On guardare les pcisions
for e in range(0, len(dataEC)):  #Per cada brain reagon
    Clean_Data_BR = []
    New_listnames_BR = []
    Possitions_BR= []
    for i in range(0, len(dataEC[e])): #Per cada Band
        Clean_Data_Band = []
        New_listnames_Band = []
        Possitions_Band = []
        df=pd.DataFrame(dataEC[e][i])  #Pas to Data frame
        df['z_score']=stats.zscore(df) #Get z-score
        for r in df['z_score']:        #Per cada linea en z-score
            if -2.5 < r > 2.5 :  #Si el z-score es major que 2.5
                rows = list(df['z_score'][df['z_score'] == float(r)].index) #Select la posicio que tingui el z-score alt
                text = str(rows) #El guardem en string
                Nombredada = df[0][int(text[1:-1])]  #busquem en la pocicio quina dada te
                Posicio = dataEC[e][i].index(Nombredada) #busquem en dataEC quina posicio te la dada
                Clean_Data_Band.append(dataEC[e][i][Posicio])
                Possitions_Band.append(Posicio)
                Nompacient = listEC_Names[Posicio] #busquem en aquella posicio el nom del pacient
                New_listnames_Band.append(Nompacient)
        Clean_Data_BR.append(Clean_Data_Band)
        New_listnames_BR.append(New_listnames_Band)
        Possitions_BR.append(Possitions_Band)
    Clean_Data.append(Clean_Data_BR)
    New_listnames.append(New_listnames_BR)
    Possitions.append(Possitions_BR)

In [6]:
#
Names_New = dabeNames_BR_Bads_BP.copy()
Dadas_New = dataEC.copy()
for e in range(0, len(dataEC)):  #Per cada brain reagon
    for i in range(0, len(dataEC[e])): #Per cada Band
        df=pd.DataFrame(dataEC[e][i])  #Pas to Data frame
        df['z_score']=stats.zscore(df) #Get z-score
        for r in df['z_score']:        #Per cada linea en z-score
            if -2.5 < r > 2.5 :  #Si el z-score es major que 2.5
                rows = list(df['z_score'][df['z_score'] == float(r)].index) #Select la posicio que tingui el z-score alt
                text = str(rows) #El guardem en string
                Nombredada = df[0][int(text[1:-1])]  #busquem en la pocicio quina dada te
                Posicio = dataEC[e][i].index(Nombredada) #busquem en dataEC quina posicio te la dada
                del Dadas_New[e][i][Posicio]
                del Names_New[e][i][Posicio]

In [7]:
#Name of the EC files in name order.
with open("Listoflist_no_outliersEC.txt", "wb") as fp:   #Pickling
    pickle.dump(Names_New, fp)

with open("Dades_no_outliersEC.txt", "wb") as fp:   #Pickling
    pickle.dump(Dadas_New, fp)

#Name of the EC files ordered in the name file.
with open("Listoflist_outliersEC.txt", "wb") as fp:   #Pickling
    pickle.dump(New_listnames, fp)

with open("Dades_outliersEC.txt", "wb") as fp:   #Pickling
    pickle.dump(Clean_Data, fp)